In [14]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

# 1) Buscar o dataset Iris (id=53)
iris = fetch_ucirepo(id=53)

# 2) Converter para DataFrames pandas
X = iris.data.features.copy()   # features (pandas DataFrame)
y = iris.data.targets.copy()    # targets (pandas DataFrame)
df = pd.concat([X, y], axis=1)
df.columns = [c.strip().lower().replace(' (cm)', '').replace(' ', '_') for c in df.columns]
df.rename(columns={'class': 'target_name'}, inplace=True)
df['y'] = (df['target_name'] == 'Iris-setosa').astype(int)


# 3) discretizar APENAS a 1ª coluna (sepal_length) em low/medium/high por quantis 33%/66%
q1, q2 = df['sepal_length'].quantile([1/3, 2/3])
df['sepal_length_disc'] = pd.cut(df['sepal_length'],
                                 bins=[-np.inf, q1, q2, np.inf],
                                 labels=['low','medium','high'])
print(df)

def entropy_binary(y_series):
    """Entropia de um alvo binário (1=setosa, 0=não-setosa)."""
    p = y_series.mean() #proporção de 1's porque e um conjunto binário
    if p == 0 or p == 1: #no caso de o conjunto ser todo de uma classe
        return 0.0 
    return -(p*np.log2(p) + (1-p)*np.log2(1-p))

H_S = entropy_binary(df['y'])
print("Entropia(S) conjunto total", round(H_S, 4))

H_low    = entropy_binary(df[df['sepal_length_disc']=='low']['y'])
H_medium = entropy_binary(df[df['sepal_length_disc']=='medium']['y'])
H_high   = entropy_binary(df[df['sepal_length_disc']=='high']['y'])

print("Entropia(S_low):   ", round(H_low, 4))
print("Entropia(S_medium):", round(H_medium, 4))
print("Entropia(S_high):  ", round(H_high, 4))

n = len(df)
n_low    = len(df[df['sepal_length_disc']=='low'])
n_medium = len(df[df['sepal_length_disc']=='medium'])
n_high   = len(df[df['sepal_length_disc']=='high'])

weighted = (n_low/n)*H_low + (n_medium/n)*H_medium + (n_high/n)*H_high
gain = H_S - weighted

print("Gain(S, sepal_length):", round(gain, 4))



     sepal_length  sepal_width  petal_length  petal_width     target_name  y  \
0             5.1          3.5           1.4          0.2     Iris-setosa  1   
1             4.9          3.0           1.4          0.2     Iris-setosa  1   
2             4.7          3.2           1.3          0.2     Iris-setosa  1   
3             4.6          3.1           1.5          0.2     Iris-setosa  1   
4             5.0          3.6           1.4          0.2     Iris-setosa  1   
..            ...          ...           ...          ...             ... ..   
145           6.7          3.0           5.2          2.3  Iris-virginica  0   
146           6.3          2.5           5.0          1.9  Iris-virginica  0   
147           6.5          3.0           5.2          2.0  Iris-virginica  0   
148           6.2          3.4           5.4          2.3  Iris-virginica  0   
149           5.9          3.0           5.1          1.8  Iris-virginica  0   

    sepal_length_disc  
0              

### 2)
Fazendo o ganho da partiçao sugerida no inicio obtivemos um valor de 0.5587 o que nao e um valor ideal, que significa que a partição não e ideal mas pode ajudar na diferenciaçao dos dois conjuntos em estudo.

### 3)

In [ ]:

# Calculo dos ganhos

# lista com as quatro features
feature_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

# criar um dicionário para guardar os ganhos
gains = {}

for col in feature_cols:
    # 1. discretizar a coluna em low/medium/high
    q1, q2 = df[col].quantile([1/3, 2/3])
    df[col + '_disc'] = pd.cut(df[col],
                               bins=[-np.inf, q1, q2, np.inf],
                               labels=['low', 'medium', 'high'])
    
    # 2. calcular entropias dos subconjuntos
    H_S = entropy_binary(df['y'])
    weighted = 0.0
    
    for value in ['low', 'medium', 'high']:
        subset = df[df[col + '_disc'] == value]
        H_subset = entropy_binary(subset['y'])
        weighted += (len(subset) / len(df)) * H_subset
    
    # 3. calcular ganho
    gain = H_S - weighted
    gains[col] = gain

# ===== Mostrar resultados =====
print("\nGanho de informação por feature:")
for feature, gain in gains.items():
    print(f"{feature:15s}: {gain:.4f}")

# ===== Mostrar qual é a melhor coluna =====
best_feature = max(gains, key=gains.get)
print(f"\nA melhor feature para dividir o conjunto é: {best_feature} com ganho {gains[best_feature]:.4f}")



Ganho de informação por feature:
sepal_length   : 0.5587
sepal_width    : 0.3081
petal_length   : 0.9183
petal_width    : 0.9183

A melhor feature para dividir o conjunto é: petal_length com ganho 0.9183


Resposta ao 3)

As variáveis petal_length e petal_width apresentam o maior ganho de informação (~0.92), o que demonstra que possui o maior potencial discriminativo entre Iris-setosa e as restantes espécies.
Isto significa que, ao particionar os dados com base nessa variável, a incerteza sobre a classe do exemplo reduz-se quase totalmente.

### 4) 
A melhor divisão (split) é aquela que produz a maior redução da entropia, isto é, o maior ganho de informação (Information Gain – IG). Assim, o atributo com maior IG é selecionado como nó raiz, pois é o que mais contribui para diminuir a incerteza sobre a classe. No caso do conjunto Iris, o atributo petal_length apresenta o maior ganho (≈ 0,9183), sendo, portanto, o melhor critério inicial para dividir os dados e distinguir Iris-setosa das restantes espécies.
O conjunto é então dividido consoante os valores de petal_length (low, medium, high), e o processo repete-se recursivamente nos subconjuntos, escolhendo sempre o atributo mais informativo até que os nós sejam puros ou não existam mais variáveis relevantes.